In [1]:
import numpy as np
import pandas as pd
from simulation import *
from data import *
from models import *

In [2]:
seedResults = createSeedResultsDF(seeds, tourneyCompactResults)
masterCompact = createMasterCompactDF(seedResults, regSeasCompactTotals)

In [4]:
tourneySimVsActual(list(range(1985, 2020)), highSeedWins, masterCompact)

0.6383829409151488